<a href="https://colab.research.google.com/github/TomJiX/ML_start/blob/main/ML_first_try_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data treatment

Package import

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2
import random
from datetime import datetime
import time

Variable initialization

In [18]:
FILE_NAME="cats_n_dogs"
DATADIR="/content/train"
CATEGORIES=["dog","cat"]
IMG_SIZE=70

In [19]:
training_data=[]
def create_training_data():
    i=1
    j=0
    total_len=len(os.listdir(DATADIR))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    start_time=time.time()
    print("Data processing started at %s"%current_time)
    for img in os.listdir(DATADIR):
        try:
            img_array=cv2.imread(os.path.join(DATADIR,img), cv2.IMREAD_GRAYSCALE)
            new_array= cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
            img_category = img.split('.')[0]
            training_data.append([new_array,CATEGORIES.index(img_category)])
            print("\r Data is being processed %s/%s"%(i,total_len),end="")
            i+=1
        except Exception as e:
            j+=1
            pass
    end_time=time.time()
    print("Data processed \n %s files failed \nProcessing took %ss"%(j,int(end_time-start_time)))


In [20]:
create_training_data()
random.shuffle(training_data)

X=[]
y=[]
for features, label in training_data:
    X.append(features)
    y.append(label)

X= np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE,1)

np.save(FILE_NAME+"_features.npy",X)
np.save(FILE_NAME+"_val.npy",y)
print("Done")

Data processing started at 15:11:56
 Data is being processed 25000/25000Data processed 
 0 files failed 
Processing took 29s
Done


# Model fit

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
import numpy as np
import gc
import os
import pickle

Variable initialization

In [22]:
X = np.load("/content/cats_n_dogs_features.npy")
y=np.load("/content/cats_n_dogs_val.npy")



In [23]:
X.shape

(25000, 70, 70, 1)

In [24]:

SAVE_MODEL_NAME="cnd-cnn"

if os.path.isfile(SAVE_MODEL_NAME+".model"):
    SAVE_MODEL_NAME+=str(time.time())

X = X/255.0

dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

# dense_layers = [0]
# layer_sizes = [64]
# conv_layers = [2]

EPOCHS=10

Choose the type of NN

In [25]:
def general_fit():
    for dense_layer in dense_layers:
        for layer_size in layer_sizes:
            for conv_layer in conv_layers:
                NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
                print(NAME)

                model = Sequential()

                model.add(Conv2D(layer_size, (3, 3),activation='relu', input_shape=X.shape[1:]))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                for l in range(conv_layer-1):
                    model.add(Conv2D(layer_size, (3, 3),activation='relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Flatten())

                for _ in range(dense_layer):
                    model.add(Dense(layer_size,activation='relu'))

                model.add(Dense(10,activation='softmax'))

                tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

                model.compile(loss='sparse_categorical_crossentropy',
                            optimizer='adam',
                            metrics=['accuracy'],
                            )
                model.fit(X, y,
                        batch_size=32,
                        epochs=EPOCHS,
                        validation_split=0.3,
                        callbacks=[tensorboard])
                gc.collect()
    model.save(SAVE_MODEL_NAME+".model")


def particular_fit():
    NAME = "3conv32-64-128-1-dense"+str(time.time())
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X.shape[1:]))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes


    tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.fit(X, y,
                        batch_size=32,
                        epochs=EPOCHS,
                        validation_split=0.3,
                        callbacks=[tensorboard])

    

    gc.collect()
    model.save(SAVE_MODEL_NAME+".model")

Compute the models

In [ ]:
#particular_fit()
general_fit()

1-conv-32-nodes-0-dense-1605453239
Epoch 1/10
  1/547 [..............................] - ETA: 0s - loss: 2.2569 - accuracy: 0.0312WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
547/547 [==============================] - 3s 6ms/step - loss: 0.6149 - accuracy: 0.6610 - val_loss: 0.5615 - val_accuracy: 0.7263
Epoch 2/10
547/547 [==============================] - 3s 5ms/step - loss: 0.5211 - accuracy: 0.7460 - val_loss: 0.5578 - val_accuracy: 0.7185
Epoch 3/10
547/547 [==============================] - 3s 5ms/step - loss: 0.4827 - accuracy: 0.7673 - val_loss: 0.5439 - val_accuracy: 0.7300
Epoch 4/10
547/547 [==============================] - 3s 5ms/step - loss: 0.4535 - accuracy: 0.7894 - val_loss: 0.5355 - val_accuracy: 0.7368
Epoch 5/10
547/547 [===============

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs